<a href="https://colab.research.google.com/github/Bisheshsingh/AI-ML/blob/main/Advanced_vision_networks.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Imports

In [ ]:
import os
import torch,math
import torchvision
from itertools import chain
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
import numpy as np
import pandas as pd
import cv2
import torch.backends.cudnn as cudnn
import matplotlib.pyplot as plt
import operator
from IPython.display import HTML, display
from PIL import Image 
from sklearn.model_selection import StratifiedKFold, train_test_split
from sklearn.metrics import roc_auc_score
from torchvision.transforms import ToTensor, RandomHorizontalFlip, Resize
from albumentations import *
from albumentations.pytorch import ToTensor
from tqdm.autonotebook import tqdm
import json
import time
import warnings

warnings.filterwarnings("ignore")

# Cuda


In [ ]:
!apt-get --purge remove cuda nvidia* libnvidia-*
!dpkg -l | grep cuda- | awk '{print $2}' | xargs -n1 dpkg --purge
!apt-get remove cuda-*
!apt autoremove
!apt-get update

Get:14 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Get:15 http://security.ubuntu.com/ubuntu bionic-security/restricted amd64 Packages [725 kB]
Hit:16 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Hit:17 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Get:18 http://archive.ubuntu.com/ubuntu bionic-updates/main amd64 Packages [2,935 kB]
Get:19 http://security.ubuntu.com/ubuntu bionic-security/main amd64 Packages [2,498 kB]
Get:20 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic/main Sources [1,822 kB]
Get:21 http://security.ubuntu.com/ubuntu bionic-security/universe amd64 Packages [1,459 kB]
Get:22 http://archive.ubuntu.com/ubuntu bionic-updates/universe amd64 Packages [2,238 kB]
Get:23 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic/main amd64 Packages [934 kB]
Get:24 http://archive.ubuntu.com/ubuntu bionic-updates/restricted amd64 Packages [758 kB]
Fetched 14.5 MB in 6s (2,245 kB/s)
Reading package

In [ ]:
!wget https://developer.nvidia.com/compute/cuda/9.2/Prod/local_installers/cuda-repo-ubuntu1604-9-2-local_9.2.88-1_amd64 -O cuda-repo-ubuntu1604-9-2-local_9.2.88-1_amd64.deb
!dpkg -i cuda-repo-ubuntu1604-9-2-local_9.2.88-1_amd64.deb
!apt-key add /var/cuda-repo-9-2-local/7fa2af80.pub
!apt-get update
!apt-get install cuda-9.2

^C


In [ ]:
!pip install git+git://github.com/andreinechaev/nvcc4jupyter.git

'import warnings' failed; traceback:
Traceback (most recent call last):
  File "/usr/lib/python3.7/warnings.py", line 550, in <module>
    _processoptions(sys.warnoptions)
  File "/usr/lib/python3.7/warnings.py", line 208, in _processoptions
    _setoption(arg)
  File "/usr/lib/python3.7/warnings.py", line 224, in _setoption
    import re
  File "/usr/lib/python3.7/re.py", line 124, in <module>
    import enum
  File "/usr/lib/python3.7/enum.py", line 2, in <module>
    from types import MappingProxyType, DynamicClassAttribute
  File "/usr/lib/python3.7/types.py", line 29, in <module>
    _ag = _ag()
KeyboardInterrupt
  Cloning git://github.com/andreinechaev/nvcc4jupyter.git to /tmp/pip-req-build-7mcwvy0q
  Running command git clone -q git://github.com/andreinechaev/nvcc4jupyter.git /tmp/pip-req-build-7mcwvy0q
  Created wheel for NVCCPlugin: filename=NVCCPlugin-0.0.2-py3-none-any.whl size=4306 sha256=a94edd55bc3f4602c09f48066dc3fed0a77b828f0ea851ec8f4fee6837820138
  Stored in directory

In [ ]:
%load_ext nvcc_plugin

created output directory at /content/src
Out bin /content/result.out


# Models

## AlexNet

In [ ]:
class AlexNet(nn.Module):
    def __init__(self, num_classes=10, dropout=0.5) -> None:
        super(AlexNet,self).__init__()
        self.features = nn.Sequential(
            nn.Conv2d(3, 64, kernel_size=11, stride=4, padding=2),nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=3, stride=2),

            nn.Conv2d(64, 192, kernel_size=5, padding=2),nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=3, stride=2),

            nn.Conv2d(192, 384, kernel_size=3, padding=1),nn.ReLU(inplace=True),

            nn.Conv2d(384, 256, kernel_size=3, padding=1),nn.ReLU(inplace=True),

            nn.Conv2d(256, 256, kernel_size=3, padding=1),nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=3, stride=2),
        )
        self.avgpool = nn.AdaptiveAvgPool2d((6, 6))
        self.classifier = nn.Sequential(
            nn.Dropout(p=dropout),
            nn.Linear(6*6*256, 4096),
            nn.ReLU(inplace=True),
            nn.Dropout(p=dropout),
            nn.Linear(4096, 4096),
            nn.ReLU(inplace=True),
            nn.Linear(4096, num_classes),
        )

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        x = self.features(x)
        x = self.avgpool(x)
        x = torch.flatten(x, 1)
        x = self.classifier(x)
        return x



## VGGNet

In [ ]:
VGG_arch = {
    'VGG11': [64, 'M', 128, 'M', 256, 256, 'M', 512, 512, 'M', 512, 512, 'M'],
    'VGG13': [64, 64, 'M', 128, 128, 'M', 256, 256, 'M', 512, 512, 'M', 512, 512, 'M'],
    'VGG16': [64, 64, 'M', 128, 128, 'M', 256, 256, 256, 'M', 512, 512, 512, 'M', 512, 512, 512, 'M'],
    'VGG19': [64, 64, 'M', 128, 128, 'M', 256, 256, 256, 256, 'M', 512, 512, 512, 512, 'M', 512, 512, 512, 512, 'M'],
}

class VGGNet(nn.Module):
    def __init__(self, vgg_name='VGG16'):
        super(VGGNet, self).__init__()
        self.features = self._make_layers(VGG_arch[vgg_name])
        self.classifier =  nn.Sequential(
            nn.Linear(512, 4096),nn.ReLU(inplace=True),nn.Dropout(0.5),
            nn.Linear(4096, 4096),nn.ReLU(inplace=True),nn.Dropout(0.5),
            nn.Linear(4096, 10)
        )

    def forward(self, x):
        out = self.features(x)
        out = out.view(out.size(0), -1)
        out = self.classifier(out)
        return out

    def _make_layers(self, cfg):
        layers = []
        in_channels = 3
        for x in cfg:
            if x == 'M':
                layers += [nn.MaxPool2d(kernel_size=2, stride=2)]
            else:
                layers += [nn.Conv2d(in_channels, x, kernel_size=3, padding=1),
                           nn.BatchNorm2d(x),
                           nn.ReLU(inplace=True)]
                in_channels = x
        layers += [nn.AvgPool2d(kernel_size=1, stride=1)]
        return nn.Sequential(*layers)

## EfficientNet

In [ ]:

model_urls = {
    'efficientnet_b0': 'https://www.dropbox.com/s/9wigibun8n260qm/efficientnet-b0-4cfa50.pth?dl=1',
    'efficientnet_b1': 'https://www.dropbox.com/s/6745ear79b1ltkh/efficientnet-b1-ef6aa7.pth?dl=1',
    'efficientnet_b2': 'https://www.dropbox.com/s/0dhtv1t5wkjg0iy/efficientnet-b2-7c98aa.pth?dl=1',
    'efficientnet_b3': 'https://www.dropbox.com/s/5uqok5gd33fom5p/efficientnet-b3-bdc7f4.pth?dl=1',
    'efficientnet_b4': 'https://www.dropbox.com/s/y2nqt750lixs8kc/efficientnet-b4-3e4967.pth?dl=1',
    'efficientnet_b5': 'https://www.dropbox.com/s/qxonlu3q02v9i47/efficientnet-b5-4c7978.pth?dl=1',
    'efficientnet_b6': None,
    'efficientnet_b7': None,
}

params = {
    'efficientnet_b0': (1.0, 1.0, 224, 0.2),
    'efficientnet_b1': (1.0, 1.1, 240, 0.2),
    'efficientnet_b2': (1.1, 1.2, 260, 0.3),
    'efficientnet_b3': (1.2, 1.4, 300, 0.3),
    'efficientnet_b4': (1.4, 1.8, 380, 0.4),
    'efficientnet_b5': (1.6, 2.2, 456, 0.4),
    'efficientnet_b6': (1.8, 2.6, 528, 0.5),
    'efficientnet_b7': (2.0, 3.1, 600, 0.5),
}


class Swish(nn.Module):

    def __init__(self, *args, **kwargs):
        super(Swish, self).__init__()

    def forward(self, x):
        return x * torch.sigmoid(x)


class ConvBNReLU(nn.Sequential):

    def __init__(self, in_planes, out_planes, kernel_size, stride=1, groups=1):
        padding = self._get_padding(kernel_size, stride)
        super(ConvBNReLU, self).__init__(
            nn.ZeroPad2d(padding),
            nn.Conv2d(in_planes, out_planes, kernel_size, stride, padding=0, groups=groups, bias=False),
            nn.BatchNorm2d(out_planes),
            Swish(),
        )

    def _get_padding(self, kernel_size, stride):
        p = max(kernel_size - stride, 0)
        return [p // 2, p - p // 2, p // 2, p - p // 2]


class SqueezeExcitation(nn.Module):

    def __init__(self, in_planes, reduced_dim):
        super(SqueezeExcitation, self).__init__()
        self.se = nn.Sequential(
            nn.AdaptiveAvgPool2d(1),
            nn.Conv2d(in_planes, reduced_dim, 1),
            Swish(),
            nn.Conv2d(reduced_dim, in_planes, 1),
            nn.Sigmoid(),
        )

    def forward(self, x):
        return x * self.se(x)


class MBConvBlock(nn.Module):

    def __init__(self,
                 in_planes,
                 out_planes,
                 expand_ratio,
                 kernel_size,
                 stride,
                 reduction_ratio=4,
                 drop_connect_rate=0.2):
        super(MBConvBlock, self).__init__()
        self.drop_connect_rate = drop_connect_rate
        self.use_residual = in_planes == out_planes and stride == 1
        assert stride in [1, 2]
        assert kernel_size in [3, 5]

        hidden_dim = in_planes * expand_ratio
        reduced_dim = max(1, int(in_planes / reduction_ratio))

        layers = []
        # pw
        if in_planes != hidden_dim:
            layers += [ConvBNReLU(in_planes, hidden_dim, 1)]

        layers += [
            # dw
            ConvBNReLU(hidden_dim, hidden_dim, kernel_size, stride=stride, groups=hidden_dim),
            # se
            SqueezeExcitation(hidden_dim, reduced_dim),
            # pw-linear
            nn.Conv2d(hidden_dim, out_planes, 1, bias=False),
            nn.BatchNorm2d(out_planes),
        ]

        self.conv = nn.Sequential(*layers)

    def _drop_connect(self, x):
        if not self.training:
            return x
        keep_prob = 1.0 - self.drop_connect_rate
        batch_size = x.size(0)
        random_tensor = keep_prob
        random_tensor += torch.rand(batch_size, 1, 1, 1, device=x.device)
        binary_tensor = random_tensor.floor()
        return x.div(keep_prob) * binary_tensor

    def forward(self, x):
        if self.use_residual:
            return x + self._drop_connect(self.conv(x))
        else:
            return self.conv(x)


def _make_divisible(value, divisor=8):
    new_value = max(divisor, int(value + divisor / 2) // divisor * divisor)
    if new_value < 0.9 * value:
        new_value += divisor
    return new_value


def _round_filters(filters, width_mult):
    if width_mult == 1.0:
        return filters
    return int(_make_divisible(filters * width_mult))


def _round_repeats(repeats, depth_mult):
    if depth_mult == 1.0:
        return repeats
    return int(math.ceil(depth_mult * repeats))


class EfficientNet(nn.Module):

    def __init__(self, width_mult=1.0, depth_mult=1.0, dropout_rate=0.5, num_classes=10):
        super(EfficientNet, self).__init__()

        # yapf: disable
        settings = [
            # t,  c, n, s, k
            [1,  16, 1, 1, 3],  # MBConv1_3x3, SE, 112 -> 112
            [6,  24, 2, 2, 3],  # MBConv6_3x3, SE, 112 ->  56
            [6,  40, 2, 2, 5],  # MBConv6_5x5, SE,  56 ->  28
            [6,  80, 3, 2, 3],  # MBConv6_3x3, SE,  28 ->  14
            [6, 112, 3, 1, 5],  # MBConv6_5x5, SE,  14 ->  14
            [6, 192, 4, 2, 5],  # MBConv6_5x5, SE,  14 ->   7
            [6, 320, 1, 1, 3]   # MBConv6_3x3, SE,   7 ->   7
        ]
        # yapf: enable

        out_channels = _round_filters(32, width_mult)
        features = [ConvBNReLU(3, out_channels, 3, stride=2)]

        in_channels = out_channels
        for t, c, n, s, k in settings:
            out_channels = _round_filters(c, width_mult)
            repeats = _round_repeats(n, depth_mult)
            for i in range(repeats):
                stride = s if i == 0 else 1
                features += [MBConvBlock(in_channels, out_channels, expand_ratio=t, stride=stride, kernel_size=k)]
                in_channels = out_channels

        last_channels = _round_filters(1280, width_mult)
        features += [ConvBNReLU(in_channels, last_channels, 1)]

        self.features = nn.Sequential(*features)
        self.classifier = nn.Sequential(
            nn.Dropout(dropout_rate),
            nn.Linear(last_channels, num_classes),
        )

        # weight initialization
        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                nn.init.kaiming_normal_(m.weight, mode='fan_out')
                if m.bias is not None:
                    nn.init.zeros_(m.bias)
            elif isinstance(m, nn.BatchNorm2d):
                nn.init.ones_(m.weight)
                nn.init.zeros_(m.bias)
            elif isinstance(m, nn.Linear):
                fan_out = m.weight.size(0)
                init_range = 1.0 / math.sqrt(fan_out)
                nn.init.uniform_(m.weight, -init_range, init_range)
                if m.bias is not None:
                    nn.init.zeros_(m.bias)

    def forward(self, x):
        x = self.features(x)
        x = x.mean([2, 3])
        x = self.classifier(x)
        return x


def _efficientnet(arch, pretrained, progress, **kwargs):
    width_mult, depth_mult, _, dropout_rate = params[arch]
    model = EfficientNet(width_mult, depth_mult, dropout_rate, **kwargs)
    if pretrained:
        state_dict = load_state_dict_from_url(model_urls[arch], progress=progress)

        if 'num_classes' in kwargs and kwargs['num_classes'] != 1000:
            del state_dict['classifier.1.weight']
            del state_dict['classifier.1.bias']

        model.load_state_dict(state_dict, strict=False)
    return model



def efficientnet_b0(pretrained=False, progress=True, **kwargs):
    return _efficientnet('efficientnet_b0', pretrained, progress, **kwargs)


def efficientnet_b1(pretrained=False, progress=True, **kwargs):
    return _efficientnet('efficientnet_b1', pretrained, progress, **kwargs)


def efficientnet_b2(pretrained=False, progress=True, **kwargs):
    return _efficientnet('efficientnet_b2', pretrained, progress, **kwargs)



def efficientnet_b3(pretrained=False, progress=True, **kwargs):
    return _efficientnet('efficientnet_b3', pretrained, progress, **kwargs)



def efficientnet_b4(pretrained=False, progress=True, **kwargs):
    return _efficientnet('efficientnet_b4', pretrained, progress, **kwargs)



def efficientnet_b5(pretrained=False, progress=True, **kwargs):
    return _efficientnet('efficientnet_b5', pretrained, progress, **kwargs)



def efficientnet_b6(pretrained=False, progress=True, **kwargs):
    return _efficientnet('efficientnet_b6', pretrained, progress, **kwargs)



def efficientnet_b7(pretrained=False, progress=True, **kwargs):
    return _efficientnet('efficientnet_b7', pretrained, progress, **kwargs)

# Dataset

In [ ]:
from torch.utils.data import DataLoader
import torchvision.datasets as datasets
import torchvision.transforms as transforms
import numpy as np 
from PIL import Image
import matplotlib.pyplot as plt

class dataloader:
    def __init__(self,batch_size_train,batch_size_test):
        self.transform=transforms.Compose([
             transforms.Resize(size=224),
             transforms.RandomHorizontalFlip(),
             transforms.ToTensor(),
             transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
        ])

        self.transform2=transforms.Compose([
            transforms.Resize(size=224),
            transforms.ToTensor(),
            transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010))
        ])
        
        
        self.train=datasets.CIFAR10(root='dataset/',train=True,transform=self.transform,download=True)
        self.train_size=len(self.train)
        self.train_load=DataLoader(dataset=self.train,batch_size=batch_size_train,shuffle=True)

        self.test=datasets.CIFAR10(root='dataset/',train=False,transform=self.transform2,download=True)
        self.test_size=len(self.test)
        self.test_load=DataLoader(dataset=self.test,batch_size=batch_size_test,shuffle=True)
        
    def get_train_loader(self):
        return self.train_load
    
    def get_test_loader(self):
        return self.test_load

print(dataloader(1,1).train_size)
print(dataloader(1,1).test_size)
print(dataloader(1,1).train[0][0].shape)

  0%|          | 0/170498071 [00:00<?, ?it/s]

Extracting dataset/cifar-10-python.tar.gz to dataset/
Files already downloaded and verified
50000
Files already downloaded and verified
Files already downloaded and verified
10000
Files already downloaded and verified
Files already downloaded and verified
torch.Size([3, 32, 32])


# Description

In [ ]:
import os

description={
    'project name' : 'CIFAR10_classification',
    'project dir' : os.getcwd(),
    'total_train' : dataloader(1,1).train_size,
    'total_test' : dataloader(1,1).test_size,
    'train_batch' : 125,
    'test_batch' : 125
}

requrements=[
    'torch',
    'torchvision',
    'pickles',
    'numpy',
    'pillow'
]

def allImports():
    import torch
    import torch.nn as nn
    import torch.optim as optim
    import torch.nn.functional as F
    from torch.utils.data import DataLoader
    import torchvision.datasets as datasets
    import torchvision.transforms as transforms
    import pickle as pck

Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified


# Utils

In [ ]:
import pickle as pck
def Accuracy(loader,model,device,printval=True):
    num_correct=0
    num_samples=0
    model.eval()
    pbar=tqdm(total=description['total_test'])
    with torch.no_grad():
        for x,y in loader:
            x=x.to(device)
            y=y.to(device)
            #x=x.reshape(x.shape[0],-1)
            scores=model(x)
            _,pred=scores.max(1)
            num_correct+=(pred==y).sum()
            num_samples+=pred.size(0)
            #print(num_samples)
            pbar.update(pred.size(0))
    pbar.close()      
    model.train()
    ret=(float(num_correct/num_samples)*100,int(num_correct),int(num_samples))
    if printval:
        print_accuracy(ret, 'Test')
    return ret

def save_model(model,acc):
    name=description['project dir']+'/drive/MyDrive/'+model.__class__.__name__+'-'+str(int(acc))
    file=open(name,'wb')
    pck.dump(model, file)
    
def load_model(name,path=description['project dir']+'/drive/MyDrive/'):
    file=open(path+name,'rb')
    return pck.load(file)

def print_accuracy(tup,name):
    print('-------------------------------')
    print(name)
    print('Correct predictions:',tup[1])
    print('Total samples:',tup[2])
    print('Accuracy:',"{:.2f}".format(tup[0]))
    print('-------------------------------')

def printProgressBar (iteration, total, prefix = '', suffix = '', decimals = 1, length = 100, fill = '█', printEnd = "\r"):
    percent = ("{0:." + str(decimals) + "f}").format(100 * (iteration / float(total)))
    filledLength = int(length * iteration // total)
    bar = fill * filledLength + '-' * (length - filledLength)
    print(f'\r{prefix} |{bar}| {percent}% {suffix}', end = printEnd)
    if iteration == total: 
        print()


# Training

In [ ]:

class Train:
    def __init__(self,model,train_load,criterion,optimizer,device,scheduler,printval=True):
        self.model=model
        self.train_load=train_load
        self.criterion=criterion
        self.optimizer=optimizer
        self.device=device
        self.printval=printval
        self.scheduler=scheduler
    
    def training(self,epochs):
        pbar=tqdm(total=epochs)
        for i in range(epochs):
            self.train_model(i+1)
            self.scheduler.step()
            test_acc=Accuracy(dataloader(description['train_batch'],description['test_batch']).get_test_loader(), self.model, self.device)
            save_model(self.model,test_acc[0])
            pbar.update(1)
        return self.model
        
    def train_model(self,epoch):
        num_correct=0
        num_samples=0
        pbar=tqdm(total=description['total_train'])
        for batch_idx,(data,outputs) in enumerate(self.train_load):
            data=data.to(self.device)
            outputs=outputs.to(self.device)
            #data=data.reshape(data.shape[0],-1)

            scores=self.model(data)
           
            loss=self.criterion(scores,outputs)
        
            self.optimizer.zero_grad()
            loss.backward()
            self.optimizer.step()
            
            _,pred=scores.max(1)
            num_correct+=(pred==outputs).sum()
            num_samples+=pred.size(0)
            #print(num_samples)
            pbar.update(pred.size(0))
        pbar.close()
        ret=(float(num_correct/num_samples)*100,int(num_correct),int(num_samples))
        if self.printval:
           print_accuracy(ret, 'Epoch-> '+str(epoch)) 
        else:
            pass

#Trained Models 

In [ ]:
models=(
    'AlexNet-CIFAR10-75',
    'AlexNet-Mnist-99',
    'VGGNet-CIFAR10-92',
    'VGGNet-Mnist-99',
    'EfficientNet-Mnist-99'
)

# Implementation

In [ ]:
device='cuda'
in_channels=3
classes=10
learn_rate=1e-4
batch_size_train=description['train_batch']
batch_size_test=description['test_batch']
epochs=10
class_names=['plane','car','bird','cat','deer','dog','frog','horse','ship','truck']

model=efficientnet_b0().to(device)
#model=load_model('AlexNet-Mnist-99')
if device == 'cuda':
    model = torch.nn.DataParallel(model)
    cudnn.benchmark = True



criterion=nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learn_rate)
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=200)

data_load=dataloader(batch_size_train=batch_size_train,batch_size_test=batch_size_test)

train_model=Train(model, data_load.get_train_loader(), criterion, optimizer, device,scheduler)

train_model.training(epochs)

test_acc=Accuracy(data_load.get_test_loader(), model, device)

save_model(model,test_acc[0])



Files already downloaded and verified
Files already downloaded and verified


  0%|          | 0/10 [00:00<?, ?it/s]

  0%|          | 0/50000 [00:00<?, ?it/s]

KeyboardInterrupt: ignored